# 2. Data Preprocssing

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 2.1. Columns Variability 

In [4]:
mov_fast = pd.read_csv('../data/processed/movement_fast_stat.csv')
mov_slow = pd.read_csv('../data/processed/movement_slow_stat.csv')
traffic_fast = pd.read_csv('../data/processed/traffic_fast_stat.csv')
traffic_slow = pd.read_csv('../data/processed/traffic_slow_stat.csv')

In [5]:
def find_non_varying_variables(df):
    non_varying_columns = []
    variability_percentage = []
    
    for column in df.columns:
        unique_count = df[column].nunique()
        total_count = len(df[column])
        variability = unique_count / total_count * 100
        
        if unique_count == 1:
            non_varying_columns.append(column)
            variability_percentage.append(variability)
    
    result_df = pd.DataFrame({'Variable': non_varying_columns, 'Variability Percentage': variability_percentage})
    return result_df

In [15]:
find_non_varying_variables(mov_fast)

,Variable,Variability Percentage
0,RemoteButtons_mean,0.166667
1,RemoteButtons_std,0.166667
2,RemoteButtons_min,0.166667
3,RemoteButtons_25%,0.166667
4,RemoteButtons_50%,0.166667
...,...,...
119,Sensor2OrientationZ_min,0.166667
120,Sensor2OrientationZ_25%,0.166667
121,Sensor2OrientationZ_50%,0.166667
122,Sensor2OrientationZ_75%,0.166667


In [7]:
find_non_varying_variables(mov_slow)

,Variable,Variability Percentage
0,RemoteButtons_mean,0.166667
1,RemoteButtons_std,0.166667
2,RemoteButtons_min,0.166667
3,RemoteButtons_25%,0.166667
4,RemoteButtons_50%,0.166667
...,...,...
119,Sensor2OrientationZ_min,0.166667
120,Sensor2OrientationZ_25%,0.166667
121,Sensor2OrientationZ_50%,0.166667
122,Sensor2OrientationZ_75%,0.166667


In [8]:
find_non_varying_variables(traffic_fast)

,Variable,Variability Percentage
0,size_min,0.166667
1,size_25%,0.166667
2,size_50%,0.166667
3,size_75%,0.166667


In [9]:
find_non_varying_variables(traffic_slow)

,Variable,Variability Percentage
0,size_min,0.166667
1,size_25%,0.166667
2,size_50%,0.166667
3,size_75%,0.166667


for the movement data, the data acquired from sensor 0, 1, 2, touchbutton,and remotebuttons are constants which will have to drop from our dataset, to focus only on meangful variables, that can help us make better predicition, for the traffic data, the only constant data are the ones related to packets size, it's important first to understand why those features are constans accross all the participants/games